In [1]:
import pandas as pd
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
import matplotlib.cm as cm

In [32]:
analysis = pd.read_csv('Track_analysis.csv',index_col=0)
metadata = pd.read_csv('Track_metadata.csv',index_col=0)
musicbrainz = pd.read_csv('Track_musicbrainz.csv',index_col=0)
analysis.head()

,analysis_sample_rate,audio_md5,danceability,duration,end_of_fade_in,energy,idx_bars_confidence,idx_bars_start,idx_beats_confidence,idx_beats_start,...,key,key_confidence,loudness,mode,mode_confidence,start_of_fade_out,tempo,time_signature,time_signature_confidence,track_id
0,22050,a222795e07cd65b7a530f1346f520649,0.0,218.93179,0.247,0.0,0,0,0,0,...,1,0.736,-11.197,0,0.636,218.932,92.198,4,0.778,TRAAAAW128F429D538
0,22050,bb9771eeef3d5b204a3c55e690f52a91,0.0,148.03546,0.148,0.0,0,0,0,0,...,6,0.169,-9.843,0,0.430,137.915,121.274,4,0.384,TRAAABD128F429CF47
0,22050,fa329738005ca53715d9f7381a0d1fe3,0.0,177.47546,0.282,0.0,0,0,0,0,...,8,0.643,-9.689,1,0.565,172.304,100.070,1,0.000,TRAAADZ128F9348C2E
0,22050,43cd1abd45d5a2dda16a3c65b4963bd4,0.0,233.40363,0.000,0.0,0,0,0,0,...,0,0.751,-9.013,1,0.749,217.124,119.293,4,0.000,TRAAAEF128F4273421
0,22050,580a8fe08ef0f1c7734b84547d7a8bc7,0.0,209.60608,0.066,0.0,0,0,0,0,...,2,0.092,-4.501,1,0.371,198.699,129.738,4,0.562,TRAAAFD128F92F423A


In [23]:
print(analysis.columns)
analysis.describe()

Index(['analysis_sample_rate', 'audio_md5', 'danceability', 'duration',
       'end_of_fade_in', 'energy', 'idx_bars_confidence', 'idx_bars_start',
       'idx_beats_confidence', 'idx_beats_start', 'idx_sections_confidence',
       'idx_sections_start', 'idx_segments_confidence',
       'idx_segments_loudness_max', 'idx_segments_loudness_max_time',
       'idx_segments_loudness_start', 'idx_segments_pitches',
       'idx_segments_start', 'idx_segments_timbre', 'idx_tatums_confidence',
       'idx_tatums_start', 'key', 'key_confidence', 'loudness', 'mode',
       'mode_confidence', 'start_of_fade_out', 'tempo', 'time_signature',
       'time_signature_confidence', 'track_id'],
      dtype='object')


,analysis_sample_rate,danceability,duration,end_of_fade_in,energy,idx_bars_confidence,idx_bars_start,idx_beats_confidence,idx_beats_start,idx_sections_confidence,...,idx_tatums_start,key,key_confidence,loudness,mode,mode_confidence,start_of_fade_out,tempo,time_signature,time_signature_confidence
count,10000.0,10000.0,10000.000000,10000.000000,10000.0,10000.0,10000.0,10000.0,10000.0,10000.0,...,10000.0,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000
mean,22050.0,0.0,238.507518,0.758616,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,5.276100,0.449573,-10.485668,0.691100,0.477784,229.975465,122.915449,3.564800,0.509937
std,0.0,0.0,114.137514,1.867952,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,3.554087,0.274974,5.399788,0.462063,0.191254,112.195735,35.184412,1.266239,0.373409
min,22050.0,0.0,1.044440,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.000000,-51.643000,0.000000,0.000000,1.044000,0.000000,0.000000,0.000000
25%,22050.0,0.0,176.032200,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,2.000000,0.225000,-13.163250,0.000000,0.360000,168.856000,96.965750,3.000000,0.097750
50%,22050.0,0.0,223.059140,0.199000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,5.000000,0.469000,-9.380000,1.000000,0.487000,213.879000,120.161000,4.000000,0.551000
75%,22050.0,0.0,276.375060,0.421000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,8.000000,0.659000,-6.532500,1.000000,0.606000,266.292000,144.013250,4.000000,0.864000
max,22050.0,0.0,1819.767710,43.119000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,11.000000,1.000000,0.566000,1.000000,1.000000,1813.426000,262.828000,7.000000,1.000000


In [45]:
metadata.head()

,analyzer_version,artist_7digitalid,artist_familiarity,artist_hotttnesss,artist_id,artist_latitude,artist_location,artist_longitude,artist_mbid,artist_name,artist_playmeid,genre,idx_artist_terms,idx_similar_artists,release,release_7digitalid,song_hotttnesss,song_id,title,track_7digitalid
0,NaN,165270,0.581794,0.401998,ARD7TVE1187B99BFB1,NaN,California - LA,NaN,e77e51a5-4761-45b3-9847-2051f811e366,Casual,4479,NaN,0,0,Fear Itself,300848,0.602120,SOMZWCG12A8C13C480,I Didn't Mean To,3401791
0,NaN,1998,0.630630,0.417500,ARMJAGH1187FB546F3,35.14968,"Memphis, TN",-90.04892,1c78ab62-db33-4433-8d0b-7c8dcf1849c2,The Box Tops,22066,NaN,0,0,Dimensions,300822,NaN,SOCIWDW12A8C13D406,Soul Deep,3400270
0,NaN,290021,0.487357,0.343428,ARKRRTF1187B9984DA,NaN,NaN,NaN,7a273984-edd9-4451-9c4d-39b38f05ebcd,Sonora Santanera,13951,NaN,0,0,Las Numero 1 De La Sonora Santanera,514953,NaN,SOXVLOJ12AB0189215,Amor De Cabaret,5703798
0,NaN,19072,0.630382,0.454231,AR7G5I41187FB4CE6C,NaN,"London, England",NaN,e188a520-9cb7-4f73-a3d7-2f70c6538e92,Adam Ant,12697,NaN,0,0,Friend Or Foe,287650,NaN,SONHOTT12A8C13493C,Something Girls,3226795
0,NaN,30973,0.651046,0.401724,ARXR32B1187FB57099,NaN,NaN,NaN,c6903a2e-063c-4f91-a284-17b8f421be7b,Gob,8355,NaN,0,0,Muertos Vivos,611336,0.604501,SOFSOCN12A8C143F5D,Face the Ashes,6795666


In [46]:
musicbrainz.head()

,idx_artist_mbtags,year
0,0,0
0,0,1969
0,0,0
0,0,1982
0,0,2007


In [44]:
parameters = analysis[['duration','end_of_fade_in','key','loudness','mode','start_of_fade_out','tempo','time_signature']]
parameters = pd.concat([parameters,metadata[['artist_7digitalid','artist_familiarity','artist_hotttnesss','artist_playmeid','release_7digitalid','song_hotttnesss','track_7digitalid']]],axis=1) 
parameters['year'] = musicbrainz['year']
#One Hot Encoding
cat_cols = ["genre"]
df_continuous = pd.get_dummies(metadata,columns=cat_cols)
df_continuous.columns



Index(['analyzer_version', 'artist_7digitalid', 'artist_familiarity',
       'artist_hotttnesss', 'artist_id', 'artist_latitude', 'artist_location',
       'artist_longitude', 'artist_mbid', 'artist_name', 'artist_playmeid',
       'idx_artist_terms', 'idx_similar_artists', 'release',
       'release_7digitalid', 'song_hotttnesss', 'song_id', 'title',
       'track_7digitalid'],
      dtype='object')

In [38]:
parameters  =parameters[(parameters != 0).all(1)]
parameters.describe()

,duration,end_of_fade_in,key,loudness,mode,start_of_fade_out,tempo,time_signature,artist_7digitalid,artist_familiarity,artist_hotttnesss,artist_latitude,artist_longitude,artist_playmeid,release_7digitalid,song_hotttnesss,track_7digitalid,year
count,1751.000000,1751.000000,1751.000000,1751.000000,1751.0,1751.000000,1751.000000,1751.000000,1751.000000,1751.000000,1751.000000,731.000000,731.000000,1751.000000,1751.000000,1110.000000,1.751000e+03,1751.000000
mean,231.953720,1.167146,5.673901,-9.920468,1.0,222.960885,124.223844,3.548258,68052.967447,0.639682,0.452298,38.501796,-62.291509,22334.964021,324217.355797,0.498688,3.611759e+06,1995.695031
std,92.450481,2.279507,3.136768,5.040094,0.0,90.944154,34.676070,1.254403,105718.024673,0.131290,0.116687,14.137812,49.929002,39730.124927,226775.156677,0.167044,2.519799e+06,12.586776
min,13.583220,0.061000,1.000000,-38.525000,1.0,13.583000,30.827000,1.000000,4.000000,0.018502,0.104102,-37.817530,-123.207900,-1.000000,92.000000,0.199238,1.305000e+03,1926.000000
25%,178.246080,0.183000,2.000000,-12.538000,1.0,171.093000,97.369000,3.000000,5782.500000,0.547938,0.373508,34.053490,-91.523820,-1.000000,133890.000000,0.375984,1.438049e+06,1991.000000
50%,221.909750,0.316000,6.000000,-8.958000,1.0,213.171000,121.260000,4.000000,20075.000000,0.627185,0.425728,39.952270,-80.606880,3093.000000,289146.000000,0.497469,3.265156e+06,1999.000000
75%,269.818320,0.920000,8.000000,-6.262500,1.0,259.767000,144.965000,4.000000,83063.500000,0.731184,0.517666,46.124700,-3.996670,21626.000000,513974.000000,0.624834,5.715394e+06,2005.000000
max,1598.197100,32.972000,11.000000,-1.493000,1.0,1588.216000,253.357000,7.000000,796501.000000,1.000000,1.082503,69.651020,151.206950,217242.000000,818083.000000,1.000000,9.038144e+06,2010.000000
